# Part 1

The small crab challenges you to a game! The crab is going to mix up some cups, and you have to predict where they'll end up.

The cups will be arranged in a circle and labeled clockwise (your puzzle input). For example, if your labeling were `32415`, there would be five cups in the circle; going clockwise around the circle from the first cup, the cups would be labeled `3`, `2`, `4`, `1`, `5`, and then back to `3` again.

Before the crab starts, it will designate the first cup in your list as the current cup. The crab is then going to do 100 moves.

Each move, the crab does the following actions:

- The crab picks up the three cups that are immediately clockwise of the current cup. They are removed from the circle; cup spacing is adjusted as necessary to maintain the circle.
- The crab selects a destination cup: the cup with a label equal to the current cup's label minus one. If this would select one of the cups that was just picked up, the crab will keep subtracting one until it finds a cup that wasn't just picked up. If at any point in this process the value goes below the lowest value on any cup's label, it wraps around to the highest value on any cup's label instead.
- The crab places the cups it just picked up so that they are immediately clockwise of the destination cup. They keep the same order as when they were picked up.
- The crab selects a new current cup: the cup which is immediately clockwise of the current cup.
For example, suppose your cup labeling were `389125467`. If the crab were to do merely 10 moves, the following changes would occur:
```
-- move 1 --
cups: (3) 8  9  1  2  5  4  6  7 
pick up: 8, 9, 1
destination: 2

-- move 2 --
cups:  3 (2) 8  9  1  5  4  6  7 
pick up: 8, 9, 1
destination: 7

-- move 3 --
cups:  3  2 (5) 4  6  7  8  9  1 
pick up: 4, 6, 7
destination: 3

-- move 4 --
cups:  7  2  5 (8) 9  1  3  4  6 
pick up: 9, 1, 3
destination: 7

-- move 5 --
cups:  3  2  5  8 (4) 6  7  9  1 
pick up: 6, 7, 9
destination: 3

-- move 6 --
cups:  9  2  5  8  4 (1) 3  6  7 
pick up: 3, 6, 7
destination: 9

-- move 7 --
cups:  7  2  5  8  4  1 (9) 3  6 
pick up: 3, 6, 7
destination: 8

-- move 8 --
cups:  8  3  6  7  4  1  9 (2) 5 
pick up: 5, 8, 3
destination: 1

-- move 9 --
cups:  7  4  1  5  8  3  9  2 (6)
pick up: 7, 4, 1
destination: 5

-- move 10 --
cups: (5) 7  4  1  8  3  9  2  6 
pick up: 7, 4, 1
destination: 3

-- final --
cups:  5 (8) 3  7  4  1  9  2  6 
```
In the above example, the cups' values are the labels as they appear moving clockwise around the circle; the current cup is marked with ( ).

After the crab is done, what order will the cups be in? Starting after the cup labeled 1, collect the other cups' labels clockwise into a single string with no extra characters; each number except 1 should appear exactly once. In the above example, after 10 moves, the cups clockwise from 1 are labeled 9, 2, 6, 5, and so on, producing 92658374. If the crab were to complete all 100 moves, the order after cup 1 would be 67384529.

Using your labeling, simulate 100 moves. What are the labels on the cups after cup 1?

In [1]:
from queue import deque

MY_INPUT = '624397158'


class Game:
    def __init__(self, initial_cups):
        self.cups = deque([int(i) for i in initial_cups])
        self.current = self.cups[0]
        self._min_cup = min(self.cups)
        self._max_cup = max(self.cups)
        self._removed = []
        
    def do_move(self):
        self.rotate_cup_to_last(self.current)
        self.remove_first_n_cups(3)
        dest = self.choose_destination()
        self.rotate_cup_to_last(dest)
        self.add_removed_cups_to_the_end()
        self.rotate_cup_to_last(self.current)
        self.choose_first_cup_as_current()
        
    def rotate_cup_to_last(self, cup):
        idx = self.cups.index(cup)
        self.cups.rotate(-idx - 1)
        
    def rotate_cup_to_first(self, cup):
        idx = self.cups.index(cup)
        self.cups.rotate(-idx)
        
    def remove_first_n_cups(self, n):
        for i in range(n):
            r = self.cups.popleft()
            self._removed.append(r)
            
    def add_removed_cups_to_the_end(self):
        self.cups.extend(self._removed)
        self._removed = []
            
    def choose_destination(self):
        d = self.current - 1
        while d in self._removed or d < self._min_cup:
            if d < self._min_cup:
                d = self._max_cup
            else:
                d -= 1
        return d
        
    def choose_first_cup_as_current(self):
        self.current = self.cups[0]
        
    def result(self):
        self.rotate_cup_to_first(1)
        return ''.join([str(i) for i in list(self.cups)[1:]])
    
    def show(self):
        s = ' '.join(['(%s)' %c if c == self.current else str(c) for c in self.cups])
        print(s)
        

# TEST
test_game = Game('389125467')
for i in range(10):
    test_game.do_move()
test_game.rotate_cup_to_first(1)
assert test_game.result() == '92658374'


def result1():
    game = Game(MY_INPUT)
    for i in range(100):
        game.do_move()
    return game.result()


result1()

'74698532'

In [2]:
MAX_CUP = 1000000


class Game2:
    def __init__(self, inp):
        inp = [int(i) for i in inp]
        self.current = inp[0]
        initial_cups = inp + list(range(max(inp) + 1, MAX_CUP + 1))
        self._next_cup = {}
        for cup, n_cup in zip(initial_cups[:-1], initial_cups[1:]):
            self._next_cup[cup] = n_cup
        self._next_cup[MAX_CUP] = inp[0]

    def get_next_cup(self, cup):
        return self._next_cup[cup]
        
    def set_next_cup(self, cup, next_cup):
        self._next_cup[cup] = next_cup

    def do_move(self):
        removed_cups = self.remove_cups()
        dest = self.find_destination(removed_cups)
        self.add_cups(removed_cups, dest)
        self.next_current()
        
    def remove_cups(self):
        r1 = self.get_next_cup(self.current)
        r2 = self.get_next_cup(r1)
        r3 = self.get_next_cup(r2)
        n = self.get_next_cup(r3)
        self.set_next_cup(self.current, n)
        return (r1, r2, r3)
    
    def add_cups(self, cups, add_to):
        n = self.get_next_cup(add_to)
        self.set_next_cup(add_to, cups[0])
        self.set_next_cup(cups[-1], n)
    
    def find_destination(self, removed_cups):
        d = self.current - 1
        while d in removed_cups or d == 0:
            if d == 0:
                d = MAX_CUP
            else:
                d -= 1
        return d

    def next_current(self):
        self.current = self.get_next_cup(self.current)
        

def result2():
    game = Game2(MY_INPUT)
    for i in range(10000000):
        game.do_move()
    r1 = game.get_next_cup(1)
    r2 = game.get_next_cup(r1)
    return r1 * r2


%time result2()

CPU times: user 44.7 s, sys: 120 ms, total: 44.9 s
Wall time: 46 s


286194102744